<a href="https://colab.research.google.com/github/anoushkagarg003/contract_analysis_workflow/blob/main/workflow_law_agreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.generativeai as genai
from typing import List, Optional
from pydantic import BaseModel
from langgraph.graph import StateGraph, END

c:\Users\anous\Projects\contract_analysis_workflow\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="llama3") 

C:\Users\anous\AppData\Local\Temp\ipykernel_29940\12538460.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3")


In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [4]:
class Section(BaseModel):
    content: str
    tone: Optional[str] = None
    bias: Optional[str] = None
    suggestion: Optional[str] = None

class AuditorState(BaseModel):
    document: str
    sections: Optional[List[Section]] = []

In [5]:
# Tool 1: Split Document
def split_document(state: AuditorState) -> AuditorState:
    prompts = f"""
    Break this legal/business document into logical sections.
    Respond with a JSON list of strings, where each string is a meaningful section.

    Document:
    {state.document}
    """
    response = model.invoke(prompts).content  # ✅ Use `.content` for ChatOllama
    try:
        import json
        content = response.strip().strip("```json").strip("```")
        sections = json.loads(content)
    except:
        sections = [state.document]  # fallback: no splitting
    return AuditorState(document=state.document, sections=[Section(content=s) for s in sections])


# Tool 2: Analyze Tone
def analyze_tone(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        prompt = f"Analyze the tone of this section:\n\n{section.content}\n\nRespond with 1-2 words (e.g., 'formal', 'aggressive')."
        tone = model.invoke(prompt).content.strip()
        section.tone = tone
        updated_sections.append(section)
    state.sections = updated_sections
    return state


# Tool 3: Detect Bias
def check_bias(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        prompt = f"Does the following section contain any bias? Answer clearly like 'Yes, it's biased towards X' or 'No bias detected'.\n\n{section.content}"
        bias = model.invoke(prompt).content.strip()
        section.bias = bias
        updated_sections.append(section)
    state.sections = updated_sections
    return state


# Tool 4: Suggest Rewrite
def suggest_rewrite(state: AuditorState) -> AuditorState:
    updated_sections = []
    for section in state.sections:
        if section.tone and "aggressive" in section.tone.lower() or "biased" in (section.bias or "").lower():
            prompt = f"Rewrite this section to make it more neutral and professional:\n\n{section.content}"
            suggestion = model.invoke(prompt).content.strip()
            section.suggestion = suggestion
        updated_sections.append(section)
    state.sections = updated_sections
    return state


In [6]:
workflow = StateGraph(AuditorState)
workflow.add_node("split", split_document)
workflow.add_node("tone", analyze_tone)
workflow.add_node("bias", check_bias)
workflow.add_node("rewrite", suggest_rewrite)

workflow.set_entry_point("split")
workflow.add_edge("split", "tone")
workflow.add_edge("tone", "bias")
workflow.add_edge("bias", "rewrite")
workflow.add_edge("rewrite", END)

graph = workflow.compile()

In [7]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw()
pdf_path = askopenfilename(title="Select PDF File", filetypes=[("PDF Files", "*.pdf")])

In [8]:
# doc = """
# This Agreement is irrevocable and shall not be modified under any circumstance by the Buyer.
# The Seller reserves the right to terminate at any time without cause.
# """

# input_state = AuditorState(document=doc)
# final_state = graph.invoke(input_state)

#pdf_path = input("Enter the path to your PDF file: ").strip()
pdf_text = extract_text_from_pdf(pdf_path)
input_state = AuditorState(document=pdf_text)
final_state = graph.invoke(input_state)

In [9]:
for i, section in enumerate(final_state['sections']):
    print(f"\n--- Section {i+1} ---")
    print(f"Original: {section.content}")
    print(f"Tone: {section.tone}")
    print(f"Bias: {section.bias}")
    if section.suggestion:
        print(f"Suggested Rewrite: {section.suggestion}")


--- Section 1 ---
Original:  
Page 1 
Sample Contract 
 
Contract No.___________ 
PROFESSIONAL SERVICES AGREEMENT 
 
 
THIS AGREEMENT made and entered into this _______day of                      , 20     by and between the SANTA 
CRUZ COUNTY REGIONAL TRANSPORTATION COMMISSION, hereinafter called COMMISSION, and 
________    ____, hereinafter called CONSULTANT for __________________ (services/project name).   
 
1. DUTIES.  
A. CONSULTANT agrees to exercise special skill to accomplish the following results in a manner 
reasonably satisfactory to COMMISSION: ______________________________, as specified in Exhibit 
A: Scope of Services, which by this reference is incorporated herein. 
 
B. CONSULTANT shall provide the personnel listed below to perform the above-specified services, which 
persons are hereby designated as key personnel under this Agreement.   
 
Name  
 
 
Firm 
 
 
 
Function 
 
 
 
 
 
 
 
 
 
Principal in Charge 
 
 
 
 
 
 
 
 
 
Project Manager 
 
C. No person named 